In [42]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *

from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
#sc = SparkContext()
import sparknlp
spark = sparknlp.start()

In [3]:
ls

1080412.html  531548.html  deleteIt.ipynb          Untitled1.ipynb
1650268.html  69468.html   examples.ipynb          Untitled2.ipynb
261724.html   695644.html  mini_newsgroups.tar.gz  Untitled.ipynb
3136860.html  798556.html  mycsv.csv               xmlDoc.xml
3566684.html  data/        nlp.ipynb               xml.ipynb
3963996.html  data.csv/    people.txt


In [43]:
# data loading
data = spark.sparkContext.wholeTextFiles("xmlDoc.xml")
df = data.toDF(schema=["filename", "text"]).select("text")
df.take(1)


[Row(text='<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<?xml-stylesheet type="text/xsl" href="CDA.xsl"?>\n<!--\n Title:        History and Physical\n Filename:     C-CDA_R2_History_and_Physical.xml\n\n $LastChangedDate: 2014-11-12 23:25:09 -0500 (Wed, 12 Nov 2014) $\n\n ********************************************************\n Disclaimer: This sample file contains representative data elements to represent a History and Physical.\n The file depicts a fictional character\'s health data. Any resemblance to a real person is coincidental.\n To illustrate as many data elements as possible, the clinical scenario may not be plausible.\n The data in this sample file is not intended to represent real patients, people or clinical events.\n This sample is designed to be used in conjunction with the C-CDA Clinical Notes Implementation Guide.\n ********************************************************\n -->\n<ClinicalDocument xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="

In [12]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

action = "extract"

tag = "name"
patterns = [tag]

documentNormalizer = DocumentNormalizer() \
    .setInputCols("document") \
    .setOutputCol("normalizedDocument") \
    .setAction(action) \
    .setPatterns(patterns) \
    .setReplacement(" ") \
    .setPolicy("pretty_all") \
    .setLowercase(True)

sentenceDetector = SentenceDetector() \
      .setInputCols(["normalizedDocument"]) \
      .setOutputCol("sentence")

regexTokenizer = Tokenizer() \
      .setInputCols(["sentence"]) \
      .setOutputCol("token") \
      .fit(df)

docPatternRemoverPipeline = \
  Pipeline() \
    .setStages([
        documentAssembler,
        documentNormalizer,
        sentenceDetector,
        regexTokenizer])

ds = docPatternRemoverPipeline.fit(df).transform(df)

ds.select("normalizedDocument").show(10, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|normalizedDocument                                                                                                                                                                                                                                                                                                        

In [61]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")


action = "extract"

tag = "class"
patterns = [tag]

documentNormalizer = DocumentNormalizer() \
    .setInputCols("document") \
    .setOutputCol("normalizedDocument") \
    .setAction(action) \
    .setPatterns(patterns) \
    .setReplacement(" ") \
    .setPolicy("pretty_all") \
    .setLowercase(True)
    

sentenceDetector = SentenceDetector()\
    .setInputCols(['normalizedDocument'])\
    .setOutputCol('sentence') 

regexTokenizer = Tokenizer() \
      .setInputCols(["sentence"]) \
      .setOutputCol("token") \
      .fit(df)


docPatternRemoverPipeline = \
  Pipeline() \
    .setStages([
        documentAssembler,
        documentNormalizer,
        sentenceDetector,
        regexTokenizer])

text = """
<div id="theworldsgreatest" class='my-right my-hide-small my-wide toptext' style="font-family:'Segoe UI',Arial,sans-serif">
                <class use="L">
                    <!-- L is "Legal" from HL7 EntityNameUse 2.16.840.1.113883.5.45 -->
                    <given>Isabella</given>
                    <given>Isa</given>
                    <!-- CL is "Call me" from HL7 EntityNamePartQualifier 2.16.840.1.113883.5.43 -->
                    <family>Jones</family>
                </class>
</div>"""
data = spark.createDataFrame([[text]]).toDF("text")
data.show(10, False)
pipelineModel = docPatternRemoverPipeline.fit(data)
result = pipelineModel.transform(data)
#result.selectExpr("normalizedDocument").show(5, False)
result.selectExpr("token").show(5, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

# 